In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [24]:
sentence = 'Bank is a financial institution.My friend works in a bank.He opened me a bank account.He even gave me a lift in his a car.'
sentences = nltk.sent_tokenize(sentence.lower())
for s in sentences:
    print(s)

bank is a financial institution.my friend works in a bank.he opened me a bank account.he even gave me a lift in his a car.


In [25]:
def removePunctuation(sentences):
    tt=""
    sentences2=[]
    for x in sentences:
        tokenizer = RegexpTokenizer('\w+')
        text2=tokenizer.tokenize(x)
        cnt=1
        for x2 in text2:
            if cnt==1:
                tt+=x2
                cnt=0
            else:
                tt+=" "+x2
        sentences2.append(tt)
        tt=""
    return sentences2

In [26]:
def stopWords(sentences2):
    stop_words = set(stopwords.words("english"))
    context_tab=[]
    for sentence in sentences2:
        words = nltk.word_tokenize(sentence)
        without_stop_words = [word for word in words if not word in stop_words]
        context_tab.append(without_stop_words)
    return context_tab

In [27]:
def lemmatization(context_tab):
    lemma=[]
    wl=WordNetLemmatizer()
    for x in context_tab:
        m2=[]
        for x2 in x:
            x3=wl.lemmatize(x2,wn.VERB)
            x3=wl.lemmatize(x3,wn.NOUN)
            x3=wl.lemmatize(x3,wn.ADJ)
            x3=wl.lemmatize(x3,wn.ADV)
            m2.append(x3)
        lemma.append(m2)
    return lemma

In [28]:
def tagPos(lemma):
    pos=[]
    for n in lemma:
        pos.append(nltk.pos_tag(n))
    return pos

In [29]:
def wordOfInterest(pos):
    wn_pos=['NN','VB','JJ','JJR','JJS','NNP','VBG','RB','VBD','VBP']

    woi1=[]

    for x in pos:
        arr=[]
        for y in x:
            if y[1] in wn_pos:
                arr.append(y)
        woi1.append(arr)
    woi=[]


    for i in woi1:
        arr2=[]
        for j in i:

            if j[1]=='VBD' or j[1]=='VB' or j[1]=='VBP':
                tup=(j[0],'v')
                arr2.append(tup)
            elif j[1]=='VBG':
                tup=(j[0],'n')
                arr2.append(tup)
            elif j[1]=='NN' or j[1]=='NNP':
                tup=(j[0],'n')
                arr2.append(tup)
            elif j[1]== 'JJ' or j[1]=='JJR' or j[1]=='JJS':
                tup=(j[0],'a')
                arr2.append(tup)
            elif j[1]=='RB':
                tup=(j[0],'r')
                arr2.append(tup)
        woi.append(arr2)

    return woi

In [30]:
sentence_without_punctation = removePunctuation(sentences)
sentence_without_stopwords = stopWords(sentence_without_punctation)
lemmas = lemmatization(sentence_without_stopwords)
pos = tagPos(lemmas)
change_to_pos = wordOfInterest(pos)

In [31]:
w=change_to_pos[0][1][0]
tag=change_to_pos[0][1][1]
print(w, tag)

financial a


In [32]:
def cosineSimilarity(X,Y):
    # tokenization
    X_list = word_tokenize(X)
    Y_list = word_tokenize(Y)

    # sw contains the list of stopwords
    sw = stopwords.words('english')
    l1 =[];l2 =[]

    # remove stop words from string
    X_set = {w for w in X_list if not w in sw}
    Y_set = {w for w in Y_list if not w in sw}

    # form a set containing keywords of both strings
    rvector = X_set.union(Y_set)
    for w in rvector:
        if w in X_set: l1.append(1) # create a vector
        else: l1.append(0)
        if w in Y_set: l2.append(1)
        else: l2.append(0)
    c = 0

    # cosine formula
    for i in range(len(rvector)):
            c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    return cosine

In [33]:
def WSD(woi):
    outcome = []
    lineNo=0
    for line in woi:
        wordSenseAr=[]
        for word in line:
            senseAr=[]
            temp=[]
            senseAr = wn.synsets(word[0], word[1])
            for i in senseAr:
                score=0
                count=0
                for j in i.examples():
                    score=score+cosineSimilarity(j,sentences[lineNo])
                    count=count+1
                if count ==0:
                    score=0
                else:
                    score = score/count
                temp.append((i,score))
            temp = sorted(temp, key=lambda tup: tup[1],reverse=True)
            wordSenseAr.append(temp[0:int(len(temp)/2+1)])
        outcome.append(wordSenseAr)
    print(outcome)
    print('\n')